## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.71,66,56,10.27,broken clouds
1,1,Khatanga,RU,71.9667,102.5000,-29.15,100,99,4.09,overcast clouds
2,2,Kieta,PG,-6.2167,155.6333,79.11,83,99,1.05,overcast clouds
3,3,Vaini,TO,-21.2000,-175.2000,73.56,100,75,3.44,light intensity drizzle
4,4,Barrow,US,71.2906,-156.7887,-7.58,71,40,16.11,haze


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.71,66,56,10.27,broken clouds
2,2,Kieta,PG,-6.2167,155.6333,79.11,83,99,1.05,overcast clouds
3,3,Vaini,TO,-21.2000,-175.2000,73.56,100,75,3.44,light intensity drizzle
12,12,Hilo,US,19.7297,-155.0900,76.73,80,100,10.36,light rain
15,15,Cabedelo,BR,-6.9811,-34.8339,75.87,76,61,13.00,light rain
34,34,Avera,US,33.1940,-82.5271,75.54,28,0,11.23,clear sky
36,36,Atar,MR,20.5169,-13.0499,77.85,16,0,9.86,clear sky
42,42,Port Hedland,AU,-20.3167,118.5667,77.05,83,0,6.91,clear sky
43,43,Chuy,UY,-33.6971,-53.4616,71.62,68,92,10.69,overcast clouds
50,50,Atuona,PF,-9.8000,-139.0333,79.95,72,83,21.14,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,76.71,broken clouds,-23.1203,-134.9692,
2,Kieta,PG,79.11,overcast clouds,-6.2167,155.6333,
3,Vaini,TO,73.56,light intensity drizzle,-21.2000,-175.2000,
12,Hilo,US,76.73,light rain,19.7297,-155.0900,
15,Cabedelo,BR,75.87,light rain,-6.9811,-34.8339,
34,Avera,US,75.54,clear sky,33.1940,-82.5271,
36,Atar,MR,77.85,clear sky,20.5169,-13.0499,
42,Port Hedland,AU,77.05,clear sky,-20.3167,118.5667,
43,Chuy,UY,71.62,overcast clouds,-33.6971,-53.4616,
50,Atuona,PF,79.95,broken clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,76.71,broken clouds,-23.1203,-134.9692,People ThankYou
2,Kieta,PG,79.11,overcast clouds,-6.2167,155.6333,Uruna Bay Retreat
3,Vaini,TO,73.56,light intensity drizzle,-21.2000,-175.2000,Keleti Beach Resort
12,Hilo,US,76.73,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
15,Cabedelo,BR,75.87,light rain,-6.9811,-34.8339,Hotel Pigalle


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))